# 1 Детекция + верификация (3 балла)

Часть 1: Обнаружение лиц в реальном времени или на видео (3 балла)
Задача:

Используя библиотеку OpenCV и метод [YuNET](https://docs.opencv.org/4.x/d0/dd4/tutorial_dnn_face.html) (или другой нейросетевой метод обнаружения лиц на ваше усмотрение), реализуйте обнаружение лиц на видео (или веб-камере).
Программа должна обрабатывать кадры с вебкамеры и корректно распознавать ваше лицо. Ваше лицо должно быть выделено зелёным прямоугольником, а чужие - красными.

In [26]:
! pip install opencv-contrib-python

^C


In [43]:
import cv2
import numpy as np

In [44]:
def get_reference_face(detector, frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector.detect(frame)

    if faces[1] is not None:
        return faces[1][0][:4] 
    
    return None

def color_frame(frame, faces, reference_face):
    if faces is not None:
        for face in faces:
            x, y, dx, dy = map(int, face[:4])

            diff = 50
            if reference_face is not None and np.linalg.norm(np.array(reference_face) - np.array([x, y, dx, dy])) < diff:
                color = (0, 255, 0) 

            else:
                color = (0, 0, 255)

            cv2.rectangle(frame, (x, y), (x + dx, y + dy), color, 2)


In [45]:
model_name = "face_detection_yunet_2023mar.onnx"
camera_shape = (640, 480)
thr = .9

det_model = cv2.FaceDetectorYN_create(model_name, "", camera_shape)
det_model.setScoreThreshold(thr)

In [50]:
cap = cv2.VideoCapture(0)
reference_face = None

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    faces = det_model.detect(frame)
    
    if reference_face is None:
        reference_face = get_reference_face(det_model, frame)
    
    color_frame(frame, faces[1], reference_face)
    cv2.imshow('Face Verification', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [31]:
# https://colab.research.google.com/github/spmallick/learnopencv/blob/master/Face-Detection-Ultimate-Guide/face_detection_inference_combined.ipynb#scrollTo=EOVSxbI2wSVG - source